### github
https://github.com/rasbt/LLMs-from-scratch/blob/b12dbf6c682c2ce971ff2256f243f2e55285f974/ch03/01_main-chapter-code/ch03.ipynb

In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [16]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_outはnum_headsで割りける値を設定"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        # 重みパラメータの初期化
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # headの結合
        self.out_proj = nn.Linear(d_out,d_out)
        # ドロップアウト
        self.dropout = nn.Dropout(dropout)
        # マスク（causal attention）: context_lengthは最大長
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length),diagonal=1) # 上三角行列（対角は０）で１の部分がマスクされる
        )

    def forward(self, x):
        # バッチ、トークン数、埋め込み次元
        b, num_tokens, d_in = x.shape

        # クエリ、キー、バリュー
        ## テンソルの形状：b x num_tokens x d_out ← 計算するためにd_outをnum_heads x head_dimに分解する必要あり
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        ## テンソルの形状を b x num_tokens x num_heads x head_dimに
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        ## num_tokensとhead_dimがテンソルの後ろに移動し、行列計算できるようにする
        keys = keys.transpose(1,2) # 要素番号1,2の位置を入れ替え
        values = values.transpose(1,2)
        queries = queries.transpose(1,2)

        # アテンションスコアの計算
        attn_scores = queries @ keys.transpose(2,3) # 内積計算するためにkeysの方はnum_tokensとhead_dimを入れ替えて転置
        
        # causal attention
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # マスクをトークン数で切り捨て
        attn_scores.masked_fill_(mask_bool,-torch.inf)

        # 正規化
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # ドロップアウト
        attn_weights = self.dropout(attn_weights)

        # コンテキストベクトル
        context_vec = (attn_weights @ values).transpose(1,2) # b x num_tokens x num_heads x head_dimにもどす
        ## テンソルの形状を出力に直す（各ヘッドの計算結合をマージ）
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        ## 線形射影を追加　※厳密には必要ないが多くのLLMアーキテクチャで一般的に使われているため参考として追加しているそう
        context_vec = self.out_proj(context_vec)
        return context_vec


In [17]:
batch = torch.stack((inputs,inputs),dim=0)
print(batch.shape) # 文章 x トークン数 x 埋め込み次元

torch.Size([2, 6, 3])


In [18]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [2]:
torch.triu(torch.ones(4,4),diagonal=1)

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])